In [28]:
import re
import numpy as np
import pandas as pd
import pickle
from eunjeon import Mecab
from gensim.models import Word2Vec as w2v
from sklearn.feature_extraction.text import CountVectorizer
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [29]:
#20대 국회 법안 데이터 전처리

df = pd.read_excel('20th_bills_history.xlsx')

In [30]:
#가결/부결 코딩

for i in range(len(df)):
    if df.loc[i, '처리구분'] == 1 or df.loc[i, '처리구분'] == 2 or df.loc[i, '처리구분'] == 3 or df.loc[i, '처리구분'] == 4:
        df.loc[i, '처리구분'] = 1
    elif df.loc[i, '처리구분'] == 5 or df.loc[i, '처리구분'] == 9:
        df.loc[i, '처리구분'] = 1
    elif df.loc[i, '처리구분'] == 6 or df.loc[i, '처리구분'] == 7 or df.loc[i, '처리구분'] == 8:
        df.loc[i, '처리구분'] = 0
    elif df.loc[i, '처리구분'] == 10:
        df.loc[i, '처리구분'] = 0

In [31]:
#20대 국회 소관위 21대 국회 기준으로 바꾸기
for i in range(len(df)):
    if df.loc[i, '소관위'] =='산업통상자원위원회':
        df.loc[i, '소관위'] = '산업통상자원중소벤처기업위원회'
    elif df.loc[i, '소관위'] =='미래창조과학방송통신위원회':
        df.loc[i, '소관위'] = '과학기술정보방송통신위원회'
    elif df.loc[i, '소관위'] =='교육문화체육관광위원회':
        df.loc[i, '소관위'] = '교육위원회, 문화체육관광위원회'
    elif df.loc[i, '소관위'] =='안전행정위원회':
        df.loc[i, '소관위'] = '행정안전위원회'

In [32]:
#소관위 원-핫코딩
committee_set = list(set(df['소관위']))
committee = []
for com in committee_set:
    coms = com.split(", ")
    for c in coms:
        committee.append(c)
        
committee = list(set(committee))

for i in range(len(df)):
    if df['소관위'][i].__contains__('특별위원회'):
            df.loc[i, '특별위원회'] = 1
    for com in committee:
        if df['소관위'][i].__contains__(com):
            df.loc[i, com] = 1
        else:
            df.loc[i, com] = 0

In [33]:
#문재인 정부 이전 이후로 나누기

from datetime import datetime

for i in range(len(df)):
    if df.loc[i, '접수일자'] <= datetime.fromisoformat('2017-05-10'):
        df.loc[i, 'MoonGov'] = 0
    else:
        df.loc[i, 'MoonGov'] = 1

In [34]:
#대표발의자 소속정당 구별
#여당=0 제1야당=1 제3정당(바른미래당, 정의당, 민생당 등)=2 무소속 및 기타=3

for i in range(len(df)):
    if df.loc[i, '소속정당 구분'] == '새누리당' and df.loc[i, 'MoonGov'] == 0: #박근혜 정부 때 새누리당이 여당
        df.loc[i, 'party'] = 0
    elif df.loc[i, '소속정당 구분'] == '새누리당' and df.loc[i, 'MoonGov'] == 1:
        df.loc[i, 'party'] = 1
    elif df.loc[i, '소속정당 구분'] == '더불어민주당' and df.loc[i, 'MoonGov'] == 0:
        df.loc[i, 'party'] = 1
    elif df.loc[i, '소속정당 구분'] == '더불어민주당' and df.loc[i, 'MoonGov'] == 1: #문재인 정부 때 더불어민주당이 여당
        df.loc[i, 'party'] = 0
    elif df.loc[i, '소속정당 구분'] == '국민의당' or df.loc[i, '소속정당 구분'] == '바른미래당' or df.loc[i, '소속정당 구분'] == '바른정당':
        df.loc[i, 'party'] = 2
    elif df.loc[i, '소속정당 구분'] == '민생당' or df.loc[i, '소속정당 구분'] == '민주평화당' or df.loc[i, '소속정당 구분'] == '정의당':
        df.loc[i, 'party'] = 2
    elif df.loc[i, '소속정당 구분'] == '새로운보수당':
        df.loc[i, 'party'] = 2
    elif df.loc[i, '소속정당 구분'] == '미래통합당' or df.loc[i, '소속정당 구분'] == '미래한국당' or df.loc[i, '소속정당 구분'] == '자유한국당' :
        df.loc[i, 'party'] = 1
    elif df.loc[i, '소속정당 구분'] == '무소속':
        df.loc[i, 'party'] = 3
    elif df.loc[i, '제안자'] == '위원장':
        df.loc[i, 'party'] = 4
    elif df.loc[i, '제안자'] == '정부':
        df.loc[i, 'party'] = 5
    else:
        df.loc[i, 'party'] = 3
  

In [35]:
#대표발의자 소속정당별 의석수 코딩
party_df = pd.read_csv('20대_정당별_의석수.csv') #20대 정당별 의석수 파일 불러오기

period1 = party_df[['index', 'period 1']].rename(columns = {'index': '소속정당 구분', 'period 1': 'num_seats'})
df1 = df[df['접수일자'] <= datetime.fromisoformat('2017-01-26')]
df_merged1 = pd.merge(df1, period1, how='left', on='소속정당 구분')

period2 = party_df[['index', 'period 2']].rename(columns = {'index': '소속정당 구분', 'period 2': 'num_seats'})
df0 = df[df['접수일자'] > datetime.fromisoformat('2017-01-26')]
df2 = df0[df0['접수일자'] <= datetime.fromisoformat('2017-03-17') ]
df_merged2 = pd.merge(df2, period2, how='left', on='소속정당 구분')

period3 = party_df[['index', 'period 3']].rename(columns = {'index': '소속정당 구분', 'period 3': 'num_seats'})
df0 = df[df['접수일자'] > datetime.fromisoformat('2017-03-17')]
df3 = df0[df0['접수일자'] <= datetime.fromisoformat('2017-04-28') ]
df_merged3 = pd.merge(df3, period3, how='left', on='소속정당 구분')

period4 = party_df[['index', 'period 4']].rename(columns = {'index': '소속정당 구분', 'period 4': 'num_seats'})
df0 = df[df['접수일자'] > datetime.fromisoformat('2017-04-28')]
df4 = df0[df0['접수일자'] <= datetime.fromisoformat('2017-05-04') ]
df_merged4 = pd.merge(df4, period4, how='left', on='소속정당 구분')

period5 = party_df[['index', 'period 5']].rename(columns = {'index': '소속정당 구분', 'period 5': 'num_seats'})
df0 = df[df['접수일자'] > datetime.fromisoformat('2017-05-04')]
df5 = df0[df0['접수일자'] <= datetime.fromisoformat('2017-11-09') ]
df_merged5 = pd.merge(df5, period5, how='left', on='소속정당 구분')

period6 = party_df[['index', 'period 6']].rename(columns = {'index': '소속정당 구분', 'period 6': 'num_seats'})
df0 = df[df['접수일자'] > datetime.fromisoformat('2017-11-09')]
df6 = df0[df0['접수일자'] <= datetime.fromisoformat('2018-01-16') ]
df_merged6 = pd.merge(df6, period6, how='left', on='소속정당 구분')

period7 = party_df[['index', 'period 7']].rename(columns = {'index': '소속정당 구분', 'period 7': 'num_seats'})
df0 = df[df['접수일자'] > datetime.fromisoformat('2018-01-16')]
df7 = df0[df0['접수일자'] <= datetime.fromisoformat('2018-02-13') ]
df_merged7 = pd.merge(df7, period7, how='left', on='소속정당 구분')

period8 = party_df[['index', 'period 8']].rename(columns = {'index': '소속정당 구분', 'period 8': 'num_seats'})
df0 = df[df['접수일자'] > datetime.fromisoformat('2018-02-13')]
df8 = df0[df0['접수일자'] <= datetime.fromisoformat('2018-12-18') ]
df_merged8 = pd.merge(df8, period8, how='left', on='소속정당 구분')

period9 = party_df[['index', 'period 9']].rename(columns = {'index': '소속정당 구분', 'period 9': 'num_seats'})
df0 = df[df['접수일자'] > datetime.fromisoformat('2018-12-18')]
df9 = df0[df0['접수일자'] <= datetime.fromisoformat('2019-08-16') ]
df_merged9 = pd.merge(df9, period9, how='left', on='소속정당 구분')

period10 = party_df[['index', 'period 10']].rename(columns = {'index': '소속정당 구분', 'period 10': 'num_seats'})
df10 = df[df['접수일자']  > datetime.fromisoformat('2019-08-16') ]
df_merged10 = pd.merge(df10, period10, how='left', on='소속정당 구분')

#법안 대표발의자 소속정당별 의석수 데이터 완성
df = pd.concat([df_merged1, df_merged2, df_merged3, df_merged4, df_merged5, df_merged6, df_merged7, df_merged8, df_merged9, df_merged10]) 

In [36]:
#20대 국회 하반기 데이터만 추출

df = df[df['접수일자'] >= '2018-07-13'].reset_index()

In [37]:
#법안별 주요내용이 26개 카테고리 주제와 얼마나 유사한지 유사도 계산

model = w2v.load('word2vec.model') #모델 불러오기

tagger = Mecab()

summary_re=[]
for text in df['제안내용']:
    try:
        summary_re.append(re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z]', ' ', text))
    except:
        summary_re.append('NaN')
    
cnt_words = []
summary_re3 = []

for text in summary_re:
    pos_text = tagger.pos(text)
    word_list = []
    for word in pos_text:
        word_list.append(word[0])
    cnt_words.append(len(word_list)+1)
    summary_re3.append(' '.join(word_list))
    
df['제안내용_re']=summary_re3

In [38]:
words = list()
for word in model.wv.vocab:
    words.append(word)

#주제 카테고리 dic
category = {'국회': ['의회', '국회'],
            '정당/선거': ['정당', '선거'] ,
            '안보': ['군사', '안보', '북한'], 
            '사법': ['사법', '법원', '검찰', '소송'],
            '행정' : ['행정', '지자체', '자치', '경찰', '공무원'],
            '재정' : ['재정', '예산', '회계'], 
            '중소기업' : ['중소기업', '창업', '벤처', '스타트업'],
            '에너지' : ['에너지', '수소', '전기', '가스'], 
            '부동산' : ['주택', '부동산', '주거', '임대차'], 
            '금융' : ['금융', '투자', '자본', '보험업'], 
            '자동차' : ['자동차', '승합자동차', '이륜자동차', '승용차'],
            '건설/기계/조선' : ['기계', '건설업', '건설사', '건축', '조선업', '해운업', '발주'],
            '유통/무역' : ['유통', '물류', '무역', '쇼핑몰', '마트', '백화점'], 
            'IT' : ['IT', '통신', '게임', '데이터', '인공지능', '블록체인', '클라우드'], 
            '농축산' : ['농업', '축산', '수산'],
            '복지':['복지', '연금', '빈곤', '수당'],
            '의료/보건' : ['의료', '보건', '병원', '질병', '의약품'] , 
            '도시/교통':['도시', '교통', '운전', '도로'],
            '교육' : ['교육', '학교', '대학', '유치원', '입시'], 
            '환경' : ['친환경', '오염', '저탄소', '온실가스', '기후', '수자원'], 
            '노동' : ['노동', '임금', '노동조합', '퇴직', '채용', '근로'], 
            '치안/안전' : ['치안', '범죄', '안전', '사고', '소방', '형사'],  
            '가족' : ['가족', '아동', '청소년'],
            '여성' : ['여성', '출산', '육아', '성범죄', '성희롱'], 
            '예체능' : ['예술', '영화', '음악', '전시','공연', '문화재', '체육', '방송', '언론', '스포츠', '콘텐츠']
            }


for key in category.keys():
    keyword_similar = np.zeros(shape=(len(words), ))
    
    for keyword in category[key]:
        keyword_similar_2 = []
        for word in words:
            sm = model.wv.similarity(keyword, word)
            if sm >= 0.8:
                keyword_similar_2.append(sm)
            else:
                keyword_similar_2.append(0)

        keyword_similar_2 = np.asarray(keyword_similar_2)
        keyword_similar += keyword_similar_2
    
    keyword_similar = list(keyword_similar)
    keyword_similar_df = pd.DataFrame({'words': words, 'similar' : keyword_similar})
    keyword_similar_df = keyword_similar_df.sort_values(by=['words'], axis=0)
    keyword_similar_df = keyword_similar_df.set_index('words')
    keyword_similar_df = keyword_similar_df.T

    vectorizer = CountVectorizer(vocabulary = keyword_similar_df.columns, binary=False)
    X = vectorizer.fit_transform(summary_re3)
    words_df = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names())
    words_df = words_df.T

    tdm = np.dot(keyword_similar_df, words_df)
    tdm = tdm.T
    tdm = np.ravel(tdm) #2차원 배열을 1차원으로 평평하게 만듬
    df[key] = list(tdm / cnt_words)

In [39]:
#공동발의데이터 전처리
spon = pd.read_csv('20th_sponsorship_0104.csv')
spon = spon.loc[ spon['의안번호'].isin(df['법안코드']) ].reset_index().drop(['index'],axis=1)
spon = spon[spon['구분']!='대표'].fillna(0)
spon = spon[spon['발의자당선횟수']!=0]

In [40]:
#공동발의 의원들의 평균 선수
avg_elected = spon.pivot_table(index=['의안번호'], values=['발의자당선횟수'], aggfunc=np.mean)
avg_elected = avg_elected.reset_index()

In [41]:
spon.columns

Index(['Unnamed: 0', '의안번호', '의안명', '제안일자', '처리결과', '제안자직급', '대수', '의안종류',
       '대표발의자', '대표발의자 정당', '의안제안 코드', '당선횟수', '당선지역', '한자이름', '발의자', '발의자 정당',
       '구분', '발의자당선횟수', '발의자당선지역', '발의자한자이름'],
      dtype='object')

In [42]:
#정당다양성 변수: 공동발의 의원이 모두 같은 당 소속이면 0, 아니면 1

party_set = []

bill_no = list(set(spon['의안번호']))

for no in bill_no:
    bill = spon[spon['의안번호']==no]
    parties = set(bill['발의자 정당'])
    if len(parties) >= 2:
        party_set.append(1)
    else:
        party_set.append(0)
        
diversity = pd.DataFrame({'의안번호': bill_no, '정당다양성':party_set})

In [43]:
extra_var = pd.merge(avg_elected, diversity, how='left', on='의안번호')
extra_var = extra_var.rename(columns={"발의자당선횟수": "공동발의평균선수"})
extra_var = extra_var.rename(columns={"의안번호": "법안코드"})

df = pd.merge(df, extra_var, how='left', on='법안코드')

In [44]:
#상임위 상정 여부 변수

for i in range(len(df)):
    if type(df.loc[i, '위원회 상정일']) == pd._libs.tslibs.timestamps.Timestamp:
        df.loc[i, '상임위 상정 여부'] = 1
    else:
        df.loc[i, '상임위 상정 여부'] = 0

In [45]:
df.columns

Index(['index', '의안번호', '임기대수', '법안코드', '법안명', '제안자', '소속정당', '당선횟수', '소속위1',
       '소속위2',
       ...
       '교육', '환경', '노동', '치안/안전', '가족', '여성', '예체능', '공동발의평균선수', '정당다양성',
       '상임위 상정 여부'],
      dtype='object', length=143)

In [46]:
#소관위에서 대표발의자와 다른 정당 의원이 발언한 횟수 변수화하기

hearings = pd.read_excel('hearings_count.xlsx') #회의록 데이터
hearings = hearings.rename(columns={"법안번호": "법안코드"})
hearings_df = pd.merge(hearings, df, how='left', on='법안코드')

In [47]:
hearings_df.head(5)

,대수,법안코드,법안이름,화자,직책_x,횟수,index,의안번호,임기대수,법안명,...,교육,환경,노동,치안/안전,가족,여성,예체능,공동발의평균선수,정당다양성,상임위 상정 여부
0,20,ARC_A1E8A1H2I2X1C1Q7Q4W7A0G6F1S9Q2,교육환경 보호에 관한 법률 일부개정법률안(정부),곽상도,위원,4,39.0,2017684.0,20.0,교육환경 보호에 관한 법률 일부개정법률안(정부),...,0.068216,0.05311,0.0,0.008656,0.0,0.0,0.0,NaN,NaN,1.0
1,20,ARC_A1E8A1H2I2X1C1Q7Q4W7A0G6F1S9Q2,교육환경 보호에 관한 법률 일부개정법률안(정부),곽상도,위원,4,40.0,2017684.0,20.0,교육환경 보호에 관한 법률 일부개정법률안(정부),...,0.068216,0.05311,0.0,0.008656,0.0,0.0,0.0,NaN,NaN,1.0
2,20,ARC_A1E8A1H2I2X1C1Q7Q4W7A0G6F1S9Q2,교육환경 보호에 관한 법률 일부개정법률안(정부),김현아,위원,13,39.0,2017684.0,20.0,교육환경 보호에 관한 법률 일부개정법률안(정부),...,0.068216,0.05311,0.0,0.008656,0.0,0.0,0.0,NaN,NaN,1.0
3,20,ARC_A1E8A1H2I2X1C1Q7Q4W7A0G6F1S9Q2,교육환경 보호에 관한 법률 일부개정법률안(정부),김현아,위원,13,40.0,2017684.0,20.0,교육환경 보호에 관한 법률 일부개정법률안(정부),...,0.068216,0.05311,0.0,0.008656,0.0,0.0,0.0,NaN,NaN,1.0
4,20,ARC_A1E8A1H2I2X1C1Q7Q4W7A0G6F1S9Q2,교육환경 보호에 관한 법률 일부개정법률안(정부),문희상,의장,1,39.0,2017684.0,20.0,교육환경 보호에 관한 법률 일부개정법률안(정부),...,0.068216,0.05311,0.0,0.008656,0.0,0.0,0.0,NaN,NaN,1.0


In [48]:
#회의록 발언 의원의 정당 코딩

party = pd.read_excel('party_20th.xlsx')


period8 = party[['name', 'period 8']].rename(columns={'name':'화자'})
df8 = hearings_df[hearings_df['접수일자'] <= datetime.fromisoformat('2018-12-18') ]
df_merged8 = pd.merge(df8, period8, how='left', on='화자')

period9 = party[['name', 'period 9']].rename(columns={'name':'화자'})
df0 = hearings_df[hearings_df['접수일자'] > datetime.fromisoformat('2018-12-18')]
df9 = df0[df0['접수일자'] <= datetime.fromisoformat('2019-08-16') ]
df_merged9 = pd.merge(df9, period9, how='left', on='화자')

period10 = party[['name', 'period 10']].rename(columns={'name':'화자'})
df0 = hearings_df[hearings_df['접수일자'] > datetime.fromisoformat('2019-08-16')]
df10 = df0[df0['접수일자'] <= datetime.fromisoformat('2020-05-29') ]
df_merged10 = pd.merge(df10, period10, how='left', on='화자')

In [49]:
df_merged8 = df_merged8.rename(columns={'period 8':'화자 정당'})
df_merged9 = df_merged9.rename(columns={'period 9':'화자 정당'})
df_merged10 = df_merged10.rename(columns={'period 10':'화자 정당'})

df_re = pd.concat([df_merged8, df_merged9, df_merged10])
df_re = df_re.rename(columns={'정당':'제안자 정당'}).reset_index()

In [50]:
for i in range(len(df_re)):
    if df_re.loc[i, '접수일자'] < datetime.fromisoformat('2020-05-29'): #20대 국회 하반기인 경우
        if df_re.loc[i, '소관위'] == '법제사법위원회' or df_re.loc[i, '소관위'] == '국토교통위원회' or df_re.loc[i, '소관위'] == '보건복지위원회':
            df_re.loc[i, '상임위원장 정당'] = '자유한국당'
        elif df_re.loc[i, '소관위'] == '외교통일위원회' or df_re.loc[i, '소관위'] == '환경노동위원회' or df_re.loc[i, '소관위'] == '산업통상자원중소벤처기업위원회':
            df_re.loc[i, '상임위원장 정당'] = '자유한국당'
        elif df_re.loc[i, '소관위'] == '교육위원회':
            df_re.loc[i, '상임위원장 정당'] = '자유한국당'
        elif df_re.loc[i, '소관위'] == '과학기술정보방송통신위원회' or df_re.loc[i, '소관위'] == '국방위원회' or df_re.loc[i, '소관위'] == '국회운영위원회':
            df_re.loc[i, '상임위원장 정당'] = '더불어민주당'
        elif df_re.loc[i, '소관위'] == '기획재정위원회' or df_re.loc[i, '소관위'] == '정무위원회' or df_re.loc[i, '소관위'] == '행정안전위원회':
            df_re.loc[i, '상임위원장 정당'] = '더불어민주당'
        elif df_re.loc[i, '소관위'] == '여성가족위원회' or df_re.loc[i, '소관위'] == '문화체육관광위원회':
            df_re.loc[i, '상임위원장 정당'] = '더불어민주당'
        elif df_re.loc[i, '소관위'] == '농림축산식품해양수산위원회' or df_re.loc[i, '소관위'] == '정보위원회':
            df_re.loc[i, '상임위원장 정당'] = '민생당'
    else:
        df_re.loc[i, '상임위원장 정당'] = '더불어민주당'

In [51]:
for i in range(len(df_re)):
    if df_re.loc[i, '제안자'] == '위원장': #위원장이 발의한 경우 상임위원장 정당을 제안자 정당으로 본다
        df_re.loc[i, '제안자 정당'] = df_re.loc[i, '상임위원장 정당']
        
for i in range(len(df_re)):
    if df_re.loc[i, '제안자'] == '정부':   #정부가 발의한 경우 더불어민주당(20대 국회 하반기: 문재인 정부)을 제안자 정당으로 본다
        df_re.loc[i, '제안자 정당'] = '더불어민주당'

In [52]:
del df_re['level_0']
bill_no = list(set(df_re['법안코드']))

cnt_list = []

for no in bill_no:
    bill_df = df_re[df_re['법안코드']==no].reset_index()
    cnt = 0
    for i in range(len(bill_df)):
        if bill_df['제안자 정당'][0] != bill_df.loc[i, '화자 정당'] and type(bill_df.loc[i, '화자 정당']) == str:
            cnt+=bill_df.loc[i, '횟수']
    cnt_list.append(cnt)

bill_cnt = pd.DataFrame({'법안코드':bill_no, '반대당발언횟수': cnt_list})

In [53]:
bill_df = pd.merge(df, bill_cnt, how='left', on='법안코드')

In [54]:
bill_df.to_csv('20th_bill_preprocessed.csv', encoding='utf-8-sig') #csv파일로 저장